
# Bokeh for Time Series Analysis
<hr style="border: 2px solid black;">


<img src="./images/bokeh.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">

<img src="./images/bokeh_at_ag_glance.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">
**Introduction to Bokeh**
Bokeh is an interactive visualization library for Python that targets modern web browsers for presentation.
Unlike Matplotlib, which is primarily designed for static plots, Bokeh excels at creating
interactive plots and dashboards. It can handle large datasets and streaming data,
making it suitable for real-time applications.

**Key Features of Bokeh:**

* **Interactivity:** Built-in support for zooming, panning, hovering, and other interactive tools.
* **Web-Focused:** Generates HTML and JavaScript, making it easy to embed plots in web pages.
* **High Performance:** Can handle large datasets efficiently.
* **Versatility:** Supports a wide range of plot types (lines, bars, scatter plots, etc.).

<hr style="border: 2px solid black;">


**Documentation:**

For comprehensive documentation, please refer to the official Bokeh website: [https://docs.bokeh.org/en/latest/](https://docs.bokeh.org/en/latest/)


<hr style="border: 2px solid black;">


**Lab Exercise:**

Your task is to recreate the time series analysis lab we previously conducted using Pandas,
Matplotlib, and Seaborn, but this time, utilize the Bokeh library for visualization.
This will involve:

1.  Loading and preprocessing the "AirPassengersDates.csv" dataset.
2.  Creating interactive Bokeh plots for:
    * Time series line plots
    * Bar plots of aggregated data
    * Visualizing mean and standard deviation
    * Outlier detection
    * Resampling (upsampling and downsampling)
    * Lag analysis
    * Autocorrelation

Pay close attention to Bokeh's features for interactivity (tools, hover effects) and
its handling of data sources. Aim to replicate the insights and visualizations
from the previous lab while leveraging Bokeh's strengths.

Good luck!
<hr style="border: 2px solid black;">

In [21]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt

# Affichage Bokeh en notebook
output_notebook()

Loading BokehJS ...

In [22]:
# 1. Chargement et prétraitement
df = pd.read_csv("datasets/AirPassengersDates.csv")
df.columns = ["Date", "Passengers"]  # Renommer pour enlever le #
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Création de la source Bokeh
source = ColumnDataSource(data={
    'date': df.index,
    'passengers': df['Passengers']
})

In [23]:
# 2. Time series plot
p1 = figure(x_axis_type="datetime", title="Passagers aériens dans le temps", width=800, height=300, tools="pan,wheel_zoom,box_zoom,reset,hover")
p1.line('date', 'passengers', source=source, line_width=2, color="navy")
p1.circle('date', 'passengers', source=source, fill_color="white", size=5)
p1.hover.tooltips = [("Date", "@date{%F}"), ("Passagers", "@passengers")]
p1.hover.formatters = {"@date": "datetime"}

In [24]:
# 3. Bar plot par année
df_yearly = df.resample('Y').sum()
df_yearly.index = df_yearly.index.year
bar_source = ColumnDataSource(data={
    'year': df_yearly.index.astype(str).tolist(),  # <-- .tolist() ajouté
    'passengers': df_yearly['Passengers'].values
})

p2 = figure(x_range=bar_source.data['year'], title="Passagers par année", width=800, height=300, tools="hover")
p2.vbar(x='year', top='passengers', width=0.7, source=bar_source, color="teal")
p2.hover.tooltips = [("Année", "@year"), ("Passagers", "@passengers")]

In [25]:
# 4. Moyenne et écart type
mean_val = df['Passengers'].mean()
std_val = df['Passengers'].std()

p3 = figure(x_axis_type="datetime", title="Moyenne et écart-type", width=800, height=300)
p3.line(df.index, df['Passengers'], color='gray', line_width=2)
p3.line(df.index, [mean_val]*len(df), line_dash='dashed', color='green', legend_label="Moyenne")
p3.line(df.index, [mean_val + std_val]*len(df), line_dash='dotdash', color='orange', legend_label="+1σ")
p3.line(df.index, [mean_val - std_val]*len(df), line_dash='dotdash', color='orange', legend_label="-1σ")
p3.legend.location = "top_left"

In [26]:
# 5. Détection des outliers (> 2σ)
df['Outlier'] = df['Passengers'] > mean_val + 2 * std_val
outliers = df[df['Outlier']]
out_source = ColumnDataSource(data={
    'date': outliers.index,
    'passengers': outliers['Passengers']
})

p4 = figure(x_axis_type="datetime", title="Détection des Outliers", width=800, height=300)
p4.line(df.index, df['Passengers'], color='gray')
p4.circle('date', 'passengers', size=8, color='red', source=out_source, legend_label="Outlier")
p4.legend.location = "top_left"

In [27]:
# 6. Resampling (downsample yearly, upsample weekly avec interpolation)
df_down = df['Passengers'].resample('Y').mean()
df_up = df['Passengers'].resample('W').interpolate('linear')

down_source = ColumnDataSource(data={'date': df_down.index, 'passengers': df_down.values})
up_source = ColumnDataSource(data={'date': df_up.index, 'passengers': df_up.values})

p5 = figure(x_axis_type="datetime", title="Downsampling annuel (moyenne)", width=800, height=300)
p5.line('date', 'passengers', source=down_source, line_width=2, color="purple")

p6 = figure(x_axis_type="datetime", title="Upsampling hebdomadaire (interpolation)", width=800, height=300)
p6.line('date', 'passengers', source=up_source, line_width=2, color="darkgreen")

GlyphRenderer(id='3812', ...)

In [28]:
# 7. Lag analysis (t vs t-1)
df['Lag_1'] = df['Passengers'].shift(1)
df_lag = df.dropna()
lag_source = ColumnDataSource(df_lag)

p7 = figure(title="Analyse de Lag (t vs t-1)", width=800, height=300)
p7.circle(x='Lag_1', y='Passengers', source=lag_source, size=6, color="indigo", alpha=0.6)
p7.xaxis.axis_label = "Valeur à t-1"
p7.yaxis.axis_label = "Valeur à t"

In [30]:
def autocorr(x, max_lag=36):
    n = len(x)
    mean = x.mean()
    var = x.var()
    r = np.zeros(max_lag + 1)
    r[0] = 1.0  # lag 0 is always 1

    for lag in range(1, max_lag + 1):
        r[lag] = ((x[lag:] - mean) * (x[:-lag] - mean)).mean() / var

    return r

acf = autocorr(df['Passengers'].values)
lags = np.arange(len(acf))

source = ColumnDataSource(data=dict(lags=lags, acf=acf))

p = figure(
    title='Autocorrelation Function (ACF) of Passenger Data',
    width=800,
    height=400,
    tools='pan,box_zoom,wheel_zoom,reset,save'
)

p.vbar(
    x='lags',
    top='acf',
    source=source,
    width=0.8,
    fill_color='steelblue',
    line_color='navy'
)

p.line(
    x=[0, len(acf)],
    y=[0, 0],
    line_color='black',
    line_dash='dashed'
)

ci = 1.96 / np.sqrt(len(df))
p.line(
    x=[0, len(acf)],
    y=[ci, ci],
    line_color='red',
    line_dash='dashed',
    legend_label='95% Confidence Interval'
)
p.line(
    x=[0, len(acf)],
    y=[-ci, -ci],
    line_color='red',
    line_dash='dashed'
)

p.title.text_font_size = '16pt'
p.axis.axis_label_text_font_size = '12pt'
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.axis_label = 'Lag'
p.yaxis.axis_label = 'Autocorrelation'
p.legend.location = 'top_right'

hover = HoverTool(
    tooltips=[
        ('Lag', '@lags'),
        ('Autocorrelation', '@acf{0.000}'),
    ]
)
p.add_tools(hover)

show(p)

# Affichage final Bokeh
show(column(p1, p2, p3, p4, p5, p6, p7))